# Installings

In [10]:
!pip install naftawayh
!pip install qalsadi
!pip install future>=0.16.0
!pip install pickledb>=0.9.2
!pip install alyahmor>=0.1
!pip install libqutrub>=1.2.3
!pip install naftawayh>=0.3
!pip install pyarabic>=0.6.7
!pip install tashaphyne>=0.3.4.1
!pip install arramooz-pysqlite>=0.1
!pip install py-rouge
!pip install sklearn
!pip install scipy
!pip install pandas
!pip install openpyxl

# imports

In [11]:
import os 
from matplotlib import pyplot
from math import log2
import numpy as np

import pandas as pd 
from matplotlib import pyplot as plt

from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C

from scipy.stats import pearsonr
from scipy.stats import spearmanr
from scipy import stats

import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score               #For goodness of fit (squared R)
from sklearn import linear_model #For linear fitting
from sklearn.preprocessing import PolynomialFeatures #For transforming polynomials

import qalsadi.analex as qa     #For arabic preproceeing
import naftawayh.wordtag        #For arabic preproceeing

import nltk.data
from nltk.tokenize import TreebankWordTokenizer 


from rouge_score import rouge_scorer

import itertools

from sklearn.preprocessing import PolynomialFeatures

from matplotlib import pyplot as plt

from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C
from sklearn.gaussian_process.kernels import DotProduct, WhiteKernel

# Pre-processing

In [12]:
def arabic_preprocessing (list_of_words):
    markers=[".",",","?","]",":",";","/",'،',"["]
    new_list=[]
    for i in range(len(list_of_words)):
        flag=0
        for j in range(len(markers)):
            if markers[j] in list_of_words[i] and (markers[j] != list_of_words[i]):
                new_list.append(list_of_words[i].replace(markers[j], ''))
                flag=1
        if flag==0 and (list_of_words[i] not in markers) and list_of_words[i]!="":
            new_list.append(list_of_words[i])
    return new_list

# POS tagging

In [13]:
#in this part we detremine the nouns and verbs in the arabic text
def AR_POS(Words):
    number_of_nouns=0
    number_of_verbs=0
    number_of_adverbs=0
    number_of_adjectives=0
    tagger = naftawayh.wordtag.WordTagger()
    debug=False;
    limit=1000
    nouns = []
    verbs = []
    adverbs = []
    adjectives = []
    
    for i in range((len(Words))):
    #       print "--------تحليل كلمة  ------------", word.encode('utf8');
        analyzer = qa.Analex()
        analyzer.set_debug(debug);
        result = analyzer.check_text(Words[i]);
        #print("res=",result)
        #print(repr(result[0][0]).split(",")[0].split("=")[1])
        #print(Words[i])
        if (',' not in Words[i]):
            T = repr(result[0][0]).split(",")[10].split("=")[1]
            #print(T)
            #print(Words[i])

            if 'noun' in T.lower():
                #print("Noun")
                nouns.append(Words[i])
            elif 'verb' in T.lower():
                #print("Noun")
                verbs.append(Words[i])
            elif 'unknown' in T.lower():
                if tagger.is_noun(Words[i]):
                    nouns.append(Words[i])  
                if tagger.is_verb(Words[i]):
                    verbs.append(Words[i])
    
    return nouns , verbs

# Reading the dataset

In [14]:
file_path_1 = "PublishedCorpusFileSet/SourceTexts"
source_files = os.listdir(file_path_1)                                                   #source text

file_path_2 = "PublishedCorpusFileSet/PublishedEvaluation/AutomaticEvaluation"                                  
Automatic_evaluation_files = os.listdir(file_path_2)                                                                 
                                                                         # Human evaluation for each candidate summary
file_path_3 = "PublishedCorpusFileSet/PublishedEvaluation/HumanEvaluation"                                           
Human_evaluation_files = os.listdir(file_path_3)                                                                     

file_path_4 = "PublishedCorpusFileSet/correctedSubmissions"
corrected_files = os.listdir(file_path_4)                                #candidate summary

file_path_5 = "PublishedCorpusFileSet/Arabic"                           #Referances summary 
Arabic_files = os.listdir(file_path_5)    

#print(source_files)

In [15]:
Source = []
for file in source_files:
    path= file_path_1 + "/" + file
    with open(path,encoding='utf8') as fp:
        source = fp.read()
        fp.closed
    Source.append([source])
    
Candidate = []
for files in corrected_files:
    path= file_path_4 + "/" + files + "/Arabic"
    PATH= os.listdir(path)
    for file in PATH:
        Path= path + "/" + file
        with open(Path,encoding='utf8') as fp:
            candidate = fp.read()
            fp.closed
        Candidate.append([candidate])

Reference = []
for file in Arabic_files:
    path= file_path_5 + "/" + file
    with open(path,encoding='utf8') as fp:
        reference = fp.read()
        fp.closed
    Reference.append([reference])
       

#print(Source[0])

In [16]:
new_candidate=[]
for j in range(10):
    for i in range(j,len(Candidate),10):
        new_candidate.append(Candidate[i])
#print(len(new_candidate))
#print(new_candidate[3])


# Reading the human evaluation

In [17]:
path= file_path_3 + '/' + Human_evaluation_files[0]
with open(path,encoding='utf8') as csvFile:
    evaluation = csvFile.read()
    csvFile.closed
#print(evaluation)

In [18]:
Evaluation=[]
for i in range(6,545,2):
    Evaluation.append(int(evaluation[i]))
#print(len(Evaluation))
#print(Evaluation)

In [19]:
final_evaluation=[]

for j in range(1,269,3):
    final_evaluation.append((Evaluation[j-1]+Evaluation[j]+Evaluation[j+1])/3)
print(len(final_evaluation))
print(final_evaluation)

90
[3.6666666666666665, 4.0, 2.3333333333333335, 2.6666666666666665, 2.0, 3.0, 3.3333333333333335, 3.6666666666666665, 4.0, 2.6666666666666665, 3.0, 2.6666666666666665, 3.6666666666666665, 2.3333333333333335, 2.0, 3.6666666666666665, 3.3333333333333335, 3.3333333333333335, 4.666666666666667, 3.3333333333333335, 3.6666666666666665, 3.3333333333333335, 3.3333333333333335, 2.6666666666666665, 3.6666666666666665, 3.3333333333333335, 4.0, 4.333333333333333, 3.3333333333333335, 2.6666666666666665, 3.3333333333333335, 3.0, 2.3333333333333335, 2.6666666666666665, 4.666666666666667, 3.6666666666666665, 4.0, 3.6666666666666665, 3.6666666666666665, 3.6666666666666665, 3.0, 3.0, 4.0, 3.6666666666666665, 4.0, 3.6666666666666665, 3.3333333333333335, 3.0, 3.3333333333333335, 2.6666666666666665, 3.3333333333333335, 2.3333333333333335, 4.0, 4.0, 4.0, 2.6666666666666665, 3.0, 3.6666666666666665, 3.3333333333333335, 3.3333333333333335, 3.0, 3.3333333333333335, 4.0, 4.333333333333333, 2.6666666666666665, 

# Tokenization

In [20]:
tokenizer = TreebankWordTokenizer()
words_ip=[]
num_words_ip=0
for s in Source:
    for sentence in s:
        word=tokenizer.tokenize(sentence)
        words_ip.append(word)
print(len(words_ip))        
for x in words_ip:
    for y in x:
        num_words_ip = num_words_ip +1

words_cn=[]
num_words_cn=0
for s in new_candidate:
    for sentence in s:
        word=tokenizer.tokenize(sentence)
        words_cn.append(word)
print(len(words_cn))
for x in words_cn:
    for y in x:
        num_words_cn = num_words_cn +1
        
words_rf=[]
num_words_rf=0
for s in Reference:
    for sentence in s:
        word=tokenizer.tokenize(sentence)
        words_rf.append(word)
print(len(words_rf))        
for x in words_rf:
    for y in x:
        num_words_rf = num_words_rf +1
#print(words_rf)        
#print(num_words_ip)
#print(num_words_cn)

90
90
30


# Get number of Nouns and Verbs in the candidate summary

In [21]:
nouns =[]
verbs=[]
list_of_number_of_nouns=[]
list_of_number_of_verbs=[]
words_cn_preprocessed=[]
#print(words_cn[8])
for i in words_cn:
    words_cn_preprocessed.append(arabic_preprocessing(i))
#print(words_cn_preprocessed[46])

for i in  (words_cn_preprocessed):
    (nouns,verbs) = AR_POS(i)
    list_of_number_of_nouns.append(len(nouns))
    list_of_number_of_verbs.append(len(verbs))

Fatal Error can't execute query: file: wordfrequencydictionary
Fatal Error can't execute query: file: wordfrequencydictionary
Fatal Error can't execute query: file: wordfrequencydictionary
Fatal Error can't execute query: file: wordfrequencydictionary
Fatal Error can't execute query: file: wordfrequencydictionary
Fatal Error can't execute query: file: wordfrequencydictionary
Fatal Error can't execute query: file: wordfrequencydictionary
Fatal Error can't execute query: file: wordfrequencydictionary
Fatal Error can't execute query: file: wordfrequencydictionary
Fatal Error can't execute query: file: wordfrequencydictionary
Fatal Error can't execute query: file: wordfrequencydictionary
Fatal Error can't execute query: file: wordfrequencydictionary
Fatal Error can't execute query: file: wordfrequencydictionary
Fatal Error can't execute query: file: wordfrequencydictionary
Fatal Error can't execute query: file: wordfrequencydictionary
Fatal Error can't execute query: file: wordfrequencydic

In [22]:
print(len(list_of_number_of_nouns))
print(list_of_number_of_verbs)
#print(words_cn_preprocessed[0])

90
[47, 48, 53, 37, 44, 41, 52, 38, 49, 60, 53, 61, 48, 46, 40, 59, 56, 56, 56, 56, 61, 47, 51, 48, 49, 48, 61, 75, 59, 60, 63, 56, 53, 50, 55, 64, 55, 45, 54, 35, 49, 52, 54, 53, 47, 73, 45, 47, 55, 58, 43, 69, 49, 56, 53, 43, 60, 40, 56, 44, 60, 55, 57, 50, 47, 42, 43, 48, 47, 42, 52, 62, 45, 36, 46, 41, 37, 39, 45, 40, 46, 46, 57, 54, 51, 55, 44, 51, 48, 77]


# Get the total number of words in both Reference and candidate

In [23]:
words_ip_preprocessed=[]
for i in words_ip:
    words_ip_preprocessed.append(arabic_preprocessing(i))
print(len(words_ip_preprocessed))

words_rf_preprocessed=[]
for i in words_rf:
    words_rf_preprocessed.append(arabic_preprocessing(i))
print(len(words_rf_preprocessed))

90
30


# Get the frequency of the words in both input and candidate

In [24]:
final_ip_freq=[]
final_cn_freq=[]

wordlist_ip =[]
wordlist_cn =[]

j=0
temp=[]
for w in words_cn_preprocessed:
    #print("kk=",len(w))
    ip_freq=[]
    cn_freq=[]
    #print(w)
    #print(words_ip_preprocessed[j])
    for i in range(len(w)):
        if (w[i] in words_ip_preprocessed[j]) and (w[i] not in temp):
            ip_freq.append(words_ip_preprocessed[j].count(w[i])/len(words_ip_preprocessed[j]))
            cn_freq.append(words_cn_preprocessed[j].count(w[i])/len(words_cn_preprocessed[j]))
            temp.append(w[i])
            #print(w[i])
            #print(ip_freq)
            #print(cn_freq)
    final_ip_freq.append(ip_freq)    
    final_cn_freq.append(cn_freq) 
    j=j+1

print(len(final_cn_freq))
print(len(final_ip_freq))

90
90


In [25]:
list_of_number_of_words=[]
for i in  (words_cn_preprocessed):
    list_of_number_of_words.append(len(i))

In [26]:
print(len(list_of_number_of_nouns))
print(len(list_of_number_of_verbs))
print(len(list_of_number_of_words))

90
90
90


# Get the denisty of Nouns and Verbs

In [27]:
final_denisty_of_nouns=[]
final_denisty_of_verbs=[]
for i in range(len(list_of_number_of_words)):
    final_denisty_of_nouns.append(list_of_number_of_nouns[i]/list_of_number_of_words[i])
    final_denisty_of_verbs.append(list_of_number_of_verbs[i]/list_of_number_of_words[i])
#print(final_denisty_of_nouns)

# KL Divergence from input to summary and from summary to input and the smoothed version of KL: JS Divergence 

In [28]:
def kl_divergence(p,q):
    return sum(p[i]*log2(p[i]/q[i]) for i in range(len(p)))
def js_divergence(p, q):
    m = 0.5 * (p + q)
    return 0.5 * kl_divergence(p, m) + 0.5 * kl_divergence(q, m)

In [29]:
kl_pq=[]
kl_qp=[]
js_pq=[]

for i in range(len(final_ip_freq)):
    p=final_ip_freq[i]
    q=final_cn_freq[i]
    kl_pq.append(kl_divergence(p, q))
    kl_qp.append(kl_divergence(q, p))
    p = np.asarray(p)
    q = np.asarray(q)
    js_pq.append(js_divergence(p,q))
print(kl_pq[0])
print(len(kl_qp))
print(js_pq[0])

0.5432013208816188
90
0.057432460587732694


# Rouge

In [30]:
scorer = rouge_scorer.RougeScorer(['rouge1','rouge2','rouge3','rouge4'], use_stemmer=True)

In [31]:
#print(new_candidate)
new_reference = np.reshape(Reference, (-1, 3))
final_candidate = np.reshape(new_candidate, (-1, 9))

print(len(new_reference))
print(len(final_candidate))

score1=[]
for i in range(len(new_reference)):
    for j in range(9):
        for k in range(3):
            score1.append(scorer.score( str(new_reference[i][k]) ,str(final_candidate[i][j])))
print(len(score1))

10
10
270


# Get Fmeasure values in each Rouge

In [32]:
fmeasures_rouge1=[]
fmeasures_rouge2=[]
fmeasures_rouge3=[]
fmeasures_rouge4=[]
for i in range(270):
    x=list(score1[i].values())
    fmeasures_rouge1.append(x[0][2])
    fmeasures_rouge2.append(x[1][2])
    fmeasures_rouge3.append(x[2][2])
    fmeasures_rouge4.append(x[3][2])
#print(fmeasures_rouge1)
#print(fmeasures_rouge2)
#print(fmeasures_rouge3)
#print(fmeasures_rouge4)

In [33]:
final_fmeasures_rouge1=[]
final_fmeasures_rouge2=[]
final_fmeasures_rouge3=[]
final_fmeasures_rouge4=[]

for j in range(1,269,3):
    final_fmeasures_rouge1.append((fmeasures_rouge1[j-1]+fmeasures_rouge1[j]+fmeasures_rouge1[j+1])/3)
    final_fmeasures_rouge2.append((fmeasures_rouge2[j-1]+fmeasures_rouge2[j]+fmeasures_rouge2[j+1])/3)
    final_fmeasures_rouge3.append((fmeasures_rouge3[j-1]+fmeasures_rouge3[j]+fmeasures_rouge3[j+1])/3)
    final_fmeasures_rouge4.append((fmeasures_rouge4[j-1]+fmeasures_rouge4[j]+fmeasures_rouge4[j+1])/3)
    
#print(final_fmeasures_rouge1)
#print(final_fmeasures_rouge2)
#print(final_fmeasures_rouge3)
print(len(final_fmeasures_rouge4))

90


# Get Memo feature

In [34]:
df = pd.read_excel (r'C:\Users\hs\Desktop\MemoGrade.xlsx')
memo=df['Memo']
print(memo)

0     0.260768
1     0.804795
2     0.300892
3     0.465222
4     0.482965
        ...   
85    0.331617
86    0.420769
87    0.327406
88    0.368214
89    0.229305
Name: Memo, Length: 90, dtype: float64


In [35]:
#print(len(final_denisty_of_nouns))
#print(len(final_denisty_of_verbs))

#print(len(list_of_number_of_nouns))
#print(len(list_of_number_of_verbs))

#print(len(final_fmeasures_rouge1))
#print(len(final_fmeasures_rouge2))
#print(len(final_fmeasures_rouge3))

#print(len(kl_pq))
#print(len(kl_qp))
#print(len(js_pq))

#print(len(final_evaluation))

# Linear Regression

In [36]:
y = np.array([final_evaluation])
x = np.array([final_denisty_of_nouns, final_denisty_of_verbs,list_of_number_of_nouns ,list_of_number_of_verbs,final_fmeasures_rouge3, final_fmeasures_rouge4, kl_pq,kl_qp,js_pq ,final_fmeasures_rouge2 ,memo])

x1=np.transpose(x)
y1=np.transpose(y)

print(x1.shape)
print(y1.shape)

regr = linear_model.LinearRegression()
regr.fit(x1, y1)
y_hat_linear=regr.predict(x1)
print(y_hat_linear.shape)

(90, 11)
(90, 1)
(90, 1)


# Computing R^2 for linear regression

In [45]:
r2_score_1=r2_score(y1,regr.predict(x1))
print(r2_score_1)
print(regr.coef_)

0.2046753312250701
[[ 2.24266392e+01 -4.17054997e+01 -8.02878315e-02  1.84442001e-01
   5.40527270e+00 -3.49968495e+00 -3.27992553e+01 -3.28611854e+01
   2.86407024e+02 -7.47830878e-01 -8.38140967e-01]]


# Quadratic model

In [38]:
poly_reg = PolynomialFeatures(degree = 2)
X_poly = poly_reg.fit_transform(x1)
lin_reg_2 = linear_model.LinearRegression()
lin_reg_2.fit(X_poly, y1)
y_hat_quad=lin_reg_2.predict(X_poly)
#print(y_hat_quad)

# Computing R^2 for Quadratic regression

In [39]:
r2_score_1=r2_score(y1,lin_reg_2.predict(X_poly))
print(r2_score_1)
print(len(lin_reg_2.coef_[0]))

0.8747673991074338
78


In [40]:
y_1=list(itertools.chain(*y1))
y_estimated_linear=list(itertools.chain(*y_hat_linear))
y_estimated_quad=list(itertools.chain(*y_hat_quad))

# Calculating the correlation for linear model

In [41]:
tau, p_value = stats.kendalltau(y_1, y_estimated_linear)

print ('kendalltau correlation: %.3f' % tau ) 
print ('kendallp_value correlation: %.3f' % p_value )

corr, _ = pearsonr(y_1, y_estimated_linear) 
print('Pearsons correlation: %.3f' % corr)

corr, _ = spearmanr(y_1, y_estimated_linear) 
print('Spearmans correlation: %.3f' % corr)

kendalltau correlation: 0.330
kendallp_value correlation: 0.000
Pearsons correlation: 0.452
Spearmans correlation: 0.456


# Gaussian model

In [42]:
kernel = DotProduct() + WhiteKernel()

print (x1.shape)
print (y1.shape)

gpr = GaussianProcessRegressor(kernel=kernel,random_state=0).fit(x1, y1)
y_gauss=gpr.predict(x1)
#print(y_gauss)

(90, 11)
(90, 1)


# Calculating the correlation for gaussian model

In [43]:
y_estimated_gauss=list(itertools.chain(*y_gauss))
#Calculatin the correlation 
tau, p_value = stats.kendalltau(y_1, y_estimated_gauss)

print ('kendalltau correlation: %.3f' % tau )
print ('kendallp_value correlation: %.3f' % p_value )

corr, _ = pearsonr(y_1, y_estimated_gauss)
print('Pearsons correlation: %.3f' % corr)

corr, _ = spearmanr(y_1, y_estimated_gauss)
print('Spearmans correlation: %.3f' % corr)

kendalltau correlation: 0.224
kendallp_value correlation: 0.004
Pearsons correlation: 0.295
Spearmans correlation: 0.312


# experiment

In [44]:
#Calculatin the correlation 
tau, p_value = stats.kendalltau(y_1, kl_qp)

print ('kendalltau correlation: %.3f' % tau )
print ('kendallp_value correlation: %.3f' % p_value )

corr, _ = pearsonr(y_1, kl_qp)
print('Pearsons correlation: %.3f' % corr)

corr, _ = spearmanr(y_1, kl_qp)
print('Spearmans correlation: %.3f' % corr)

kendalltau correlation: -0.016
kendallp_value correlation: 0.840
Pearsons correlation: -0.041
Spearmans correlation: -0.027
